# Write a function for data retrival form API

In [1]:
%load_ext autoreload
%load_ext sql
%autoreload 2

import sqlite3

import matplotlib.pyplot as plt
import pandas as pd
import requests
# Import your libraries here
import os
from glob import glob
import joblib
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
from config import settings

ModuleNotFoundError: No module named 'arch'

In [2]:
# Use `dir` to list attributes
settings.alpha_api_key

'bed66634865643bd2e290f6162ba4b4990bca09f4bc2ef639eebef153ae9942cf2737b2342657cb0083cc67ae16ef37bd6d1b8e6c1c67fe234551b1cb9f8bc776a27fcb20dfa8dcc416e6e188eb2cc157a05c7f11ad778444d62c9a0312a8f8f2664bcca76f3aac097f039c2440c15e135ded8c697264c427d352b7feab29898'

In [3]:
def get_daily(ticker,output_size="full"):

    """Get daily time series of an equity from AlphaVantage API.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the equity.
    output_size : str, optional
        Number of observations to retrieve. "compact" returns the
        latest 100 observations. "full" returns all observations for
        equity. By default "full".

    Returns
    -------
    pd.DataFrame
        Columns are 'open', 'high', 'low', 'close', and 'volume'.
        All are numeric.
    """
    # Create URL 
    url = (
       "https://learn-api.wqu.edu/1/data-services/alpha-vantage/query?"
       "function=TIME_SERIES_DAILY&"
        f"symbol={ticker}&"
        f"outputsize={output_size}&"
        "datatype=json&"
        f"apikey={settings.alpha_api_key}"
    );

    # Send request to API 
    response = requests.get(url=url)

    # Extract JSON data from response 
    response_data = response.json()
    
    if "Time Series (Daily)" not in response_data.keys():
        raise Exception(
        f"Invalid API call. check the ticker symbol '{ticker}' is correct"
        )
        
    # Read data into DataFrame 
    stock_data = response_data["Time Series (Daily)"]
    df = pd.DataFrame.from_dict(stock_data,orient="index",dtype=float)

    # Convert index to `DatetimeIndex` named "date" 
    df.index =pd.to_datetime(df.index)

    # Name index "date"
    df.index.name = "date"

    # Remove numbering from columns 
    df.columns = [c.split(". ")[1] for c in df.columns]

    # Return DataFrame
    return df

In [4]:
# Test your function
df_ambuja = get_daily(ticker="AMBUJACEM.BSE")

print(df_ambuja.info())
df_ambuja.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4480 entries, 2023-03-06 to 2005-01-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    4480 non-null   float64
 1   high    4480 non-null   float64
 2   low     4480 non-null   float64
 3   close   4480 non-null   float64
 4   volume  4480 non-null   float64
dtypes: float64(5)
memory usage: 210.0 KB
None


,open,high,low,close,volume
date,,,,,
2023-03-06,396.00,404.95,381.30,385.55,2447811.0
2023-03-03,379.55,399.25,375.75,392.00,2829990.0
2023-03-02,354.40,372.10,347.25,370.85,1082223.0
2023-03-01,343.40,354.75,343.40,353.40,785471.0
2023-02-28,329.50,353.35,324.30,342.05,1146096.0


In [5]:
# Import `AlphaVantageAPI`
from data import AlphaVantageAPI

# Create instance of `AlphaVantageAPI` class
av = AlphaVantageAPI()

In [6]:
# Define Suzlon ticker symbol
ticker = "SUZLON.BSE"

# Use your `av` object to get daily data
df_suzlon = av.get_daily(ticker=ticker)

In [7]:
connection = sqlite3.connect(database=settings.db_name,check_same_thread=False)

print("connection type:", type(connection))

connection type: <class 'sqlite3.Connection'>


In [8]:
# Import class definition
from data import SQLRepository

# Create instance of class
repo = SQLRepository(connection=connection)


In [17]:
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")
response

{'transaction_successful': True, 'records_inserted': None}

In [19]:
# Import class definition
from data import SQLRepository

# Create instance of class
repo = SQLRepository(connection=connection)

# Does `repo` have a "connection" attribute?
assert  hasattr(repo,"connection")

# Is the "connection" attribute a SQLite `Connection`?
assert isinstance(repo.connection,sqlite3.Connection)

In [20]:
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")

# Does your method return a dictionary?
assert isinstance(response, dict)

# Are the keys of that dictionary correct?
assert sorted(list(response.keys())) == ["records_inserted", "transaction_successful"]